In [9]:
%run StudyHelperFile.ipynb


Variables: my_trade
           aapl_put_df
           date_string_list
           date_datetime_list
           complete_c
           my_verticle
           
Function: add_days_to_date



In [10]:
made_trade_naked = pd.DataFrame(columns = ['STRIKE','TRADE_DATE','TRADE_LAST_PRICE','DTE','EXIT_DATE','EXIT_LAST_PRICE','EXPIRE_DATE','AMOUNT_REC','AMOUNT_BUYBACK','PROFIT_PERCENT','PROFIT'])
loss_trade_naked = pd.DataFrame(columns = ['STRIKE','TRADE_DATE','TRADE_LAST_PRICE','DTE','EXIT_DATE','EXIT_LAST_PRICE','EXPIRE_DATE','AMOUNT_REC','AMOUNT_BUYBACK','PROFIT_PERCENT','PROFIT'])
next_date_list = []
next_closest_trade = complete_c[0]['QUOTE_DATE'][0]
boolean_list = []
exit_date = []

for i in range(len(complete_c)):
    each_df = complete_c[i]
    # if the closest date is equal to the first date of the df. first date of the df has to be equal since that same date is contained in other df too.
    if next_closest_trade == each_df['QUOTE_DATE'][0]: 
        win_percent = []
        amount_buyback_list = []
        profit_list = []
        # looping through each df in complete_c and calculating values
        for rows in range(len(each_df)):
            amount_received = ((each_df['P_ASK'][0] + each_df['P_BID'][0]) / 2).round(3)
            current_p_ask = each_df['P_ASK'][rows]
            current_p_bid = each_df['P_BID'][rows]
            amount_buyback = ((current_p_ask + current_p_bid) / 2).round(3)
            profit = (amount_received - amount_buyback).round(3)
            profit_percent= (((amount_received - amount_buyback).round(3)) / amount_received) * 100
            
            # These are needed to find the correct index of the value that needs to be appended in the made_trade df
            amount_buyback_list.append(amount_buyback)
            win_percent.append(profit_percent)
            profit_list.append(profit)
        
        # This boolean list is used to find out when to exit the trade. (which FIRST index of each_df is True)
        boolean_list = [True if x >= 50 else False for x in win_percent]
        if True in boolean_list:
            # Finds the first index of where each_df is true meaning that index has all the exit values.
            first_true_index = boolean_list.index(True)
            exit_date.append(each_df['QUOTE_DATE'][first_true_index])

            next_date = datetime.strptime(each_df['QUOTE_DATE'][first_true_index], '%Y-%m-%d')
            if next_date > date_datetime_list[-1]:
                break
            next_closest_trade = min([y for y in date_datetime_list if y >= next_date], key=lambda j:abs(j-next_date))
            next_closest_trade = next_closest_trade.strftime('%Y-%m-%d')
            single_made_trade_naked = pd.DataFrame([{'TRADE_DATE' : each_df['QUOTE_DATE'][0],
                                                'TRADE_LAST_PRICE': each_df['LAST_PRICE'][0],
                                                'EXIT_DATE': each_df['QUOTE_DATE'][first_true_index],
                                                'EXIT_LAST_PRICE': each_df['LAST_PRICE'][first_true_index],
                                                'STRIKE' : each_df['STRIKE'][0],
                                                'EXPIRE_DATE' : each_df['EXPIRE_DATE'][0], 
                                                'DTE' : each_df['DTE'][0], 
                                                'AMOUNT_REC' : amount_received, 
                                                'AMOUNT_BUYBACK' : amount_buyback_list[first_true_index], 
                                                'PROFIT_PERCENT' : win_percent[first_true_index], 
                                                'PROFIT' : profit_list[first_true_index]}]) # we use first_true_index of the win_percent because that index is when we exited the trade
            made_trade_naked = pd.concat([made_trade_naked, single_made_trade_naked])
            made_trade_naked = made_trade_naked.reset_index(drop = True)
        
        else:
            next_date = each_df['QUOTE_DATE'][0]
            dte = each_df['DTE'][0]
            date_expired = add_days_to_date(next_date, dte)
            if date_expired > date_datetime_list[-1]:
                break
                
            next_closest_trade = min([ y for y in date_datetime_list if y >= date_expired], key=lambda j:abs(j-date_expired))
            next_closest_trade = next_closest_trade.strftime('%Y-%m-%d')
                
            single_made_trade_naked = pd.DataFrame([{'TRADE_DATE' : each_df['QUOTE_DATE'][0],
                                                'TRADE_LAST_PRICE': each_df['LAST_PRICE'][0],
                                                'EXIT_DATE': each_df['QUOTE_DATE'].tail(1).tolist()[0],
                                                'EXIT_LAST_PRICE': each_df['LAST_PRICE'].tail(1).tolist()[0],
                                                'STRIKE' : each_df['STRIKE'][0],
                                                'EXPIRE_DATE' : each_df['EXPIRE_DATE'][0], 
                                                'DTE' : each_df['DTE'][0], 
                                                'AMOUNT_REC' : amount_received, 
                                                'AMOUNT_BUYBACK' : amount_buyback, 
                                                'PROFIT_PERCENT' : profit_percent, 
                                                'PROFIT': profit}])
            made_trade_naked = pd.concat([made_trade_naked, single_made_trade_naked])
            made_trade_naked = made_trade_naked.reset_index(drop=True)
                
made_trade_naked

,STRIKE,TRADE_DATE,TRADE_LAST_PRICE,DTE,EXIT_DATE,EXIT_LAST_PRICE,EXPIRE_DATE,AMOUNT_REC,AMOUNT_BUYBACK,PROFIT_PERCENT,PROFIT
0,112.00,2017-01-03,116.14,38.00,2017-01-10,119.11,2017-02-10,1.995,0.960,51.879699,1.035
1,115.00,2017-01-10,119.11,45.00,2017-01-25,121.92,2017-02-24,1.985,0.755,61.964736,1.230
2,118.00,2017-01-26,122.00,43.00,2017-02-01,128.75,2017-03-10,1.660,0.310,81.325301,1.350
3,125.00,2017-02-01,128.75,43.96,2017-02-07,131.51,2017-03-17,1.550,0.670,56.774194,0.880
4,128.00,2017-02-07,131.51,44.96,2017-02-14,134.93,2017-03-24,1.525,0.660,56.721311,0.865
...,...,...,...,...,...,...,...,...,...,...,...
66,347.50,2020-06-25,364.88,43.00,2020-07-06,373.81,2020-08-07,11.600,5.470,52.844828,6.130
67,355.00,2020-07-07,372.69,45.00,2020-07-20,393.43,2020-08-21,10.300,4.475,56.553398,5.825
68,126.25,2020-09-01,134.22,45.00,2020-10-16,118.96,2020-10-16,5.595,7.100,-26.899017,-1.505
69,109.00,2020-10-22,115.77,43.04,2020-11-05,118.99,2020-12-04,3.875,1.415,63.483871,2.460


In [ ]:
naked_put_sell_50_study = made_trade_naked.copy()

In [ ]:
naked_put_sell_50_study.to_csv('naked_put_sell_50_study.csv',index=False)